In [1]:
%load_ext autoreload
%autoreload 2
from ridge_utils.DataSequence import DataSequence
import pandas as pd
from os.path import dirname
import os
from tqdm import tqdm
from neuro.features import qa_questions, feature_spaces
from neuro.data import story_names, response_utils
from neuro.features.stim_utils import load_story_wordseqs, load_story_wordseqs_huge
import neuro.config
import joblib
from os.path import join

story_names_list = sorted(story_names.get_story_names(all=True))
wordseqs = load_story_wordseqs_huge(story_names_list)


class A:
    subject = 'UTS03'
    use_huge = True


args = A()

/home/chansingh/fmri/neuro/data/story_names.py:84: UserWarning: Loading all stories, ignoring subject / train_or_test
  warnings.warn('Loading all stories, ignoring subject / train_or_test')


In [ ]:
# def _get_largest_absolute_coefs(_pca, n_pcs=50, n_coefs_per_pc=50):
#     idxs_large = set()
#     for i in range(n_pcs):
#         coefs = np.abs(_pca.components_[i])
#         idxs = np.argsort(coefs)[::-1][:n_coefs_per_pc]
#         idxs_large.update(idxs)
#     idxs_large = np.array(list(idxs_large))
#     return idxs_large

for train_or_test in ['test', 'train']:
    for subject in ['UTS03', 'UTS02', 'UTS01']:
        story_names_list = story_names.get_story_names(
            subject=subject, train_or_test=train_or_test, use_huge=True)
        args.subject = subject
        for story_name in tqdm(story_names_list):
            out_file = f'{subject.lower()}/{train_or_test}/{story_name}.pkl'
            if os.path.exists(out_file):
                print('skipping', out_file)
                continue

            ngrams_list = feature_spaces.get_ngrams_list_main(
                wordseqs[story_name], num_trs_context=1)
            ngrams_list = ngrams_list[10:-5]  # apply trim
            args.pc_components = 10000
            _, resp_test, _pca, _scaler_train, _scaler_test = response_utils.get_resps_full(
                args, args.subject, [story_name], [story_name])

            # args.pc_components = -1
            # _, resp_test_full = response_utils.get_resps_full(
            # args, args.subject, [story_name], [story_name])

            # idxs_large = _get_largest_absolute_coefs(_pca)
            # resp_selected = np.hstack((resp_test, resp_test_full[:, idxs_large]))
            resp_selected = resp_test

            # print(story_name, 'shapes', resp_test.shape,
            #   resp_test_full.shape, resp_selected.shape)

            # temporal alignment
            # offset = 2
            # resp_selected = resp_selected[offset:, :]
            # ngrams_list = ngrams_list[:-offset]

            # apply convolution smoothing filter over axis 0 of resp
            # plt.plot(resp_selected[:, 0])
            # conv_filter = np.array([1/3, 1, 1/3])/(5/3)
            # resp_selected = np.apply_along_axis(
            # lambda m: np.convolve(m, conv_filter, mode='same'), axis=0, arr=resp_selected)
            # plt.plot(resp_selected[:, 0])

            # trim by 1
            # resp_selected = resp_selected[1:-1, :]
            # ngrams_list = ngrams_list[1:-1]

            assert resp_selected.shape[0] == len(
                ngrams_list), f'{resp_selected.shape[0]} != {len(ngrams_list)}'

            column_names = ['PC' + str(i) for i in range(resp_test.shape[1])]
            # + ['Vox' + str(i) for i in idxs_large]
            df = pd.DataFrame(
                resp_selected, columns=column_names, index=ngrams_list)

            # print('saving shape', df.shape)
            os.makedirs(dirname(out_file), exist_ok=True)
            df.to_pickle(out_file)
            # joblib.dump(resp_selected, f'{subject.lower()}/{story_name}_resp.pkl')
            # joblib.dump(
            # ngrams_list, f'{subject.lower()}/{story_name}_row_names_ngrams.pkl')
            # joblib.dump(
            # column_names, f'{subject.lower()}/{story_name}_column_names_fmri.pkl')

            # add answer labels
            out_file_labels = f'labels/{train_or_test}/{story_name}_labels.pkl'
            print('\tsaving', out_file_labels)
            questions = [
                'Does the input contain a number?',
                'Is time mentioned in the input?',
                'Does the sentence include dialogue?',
                'Does the input mention or describe high emotional intensity?',
                'Does the sentence mention a specific location?',
                'Is the sentence emotionally positive?',
                'Does the sentence describe a relationship between people?',
            ]
            question_answers = neuro.features.feature_spaces.get_gpt4_qa_embs_cached(
                story_name=story_name, questions=questions,
                return_ngrams=False)
            question_answers = neuro.features.feature_spaces.downsample_word_vectors(
                [story_name], {story_name: question_answers}, wordseqs)[story_name][10:-5]
            df_answers = pd.DataFrame(question_answers, columns=questions)
            os.makedirs(dirname(out_file_labels), exist_ok=True)
            df_answers = df_answers.astype(bool).to_pickle(out_file_labels)

### Example load with offsets

In [2]:
story_name = 'onapproachtopluto'
df = joblib.load(f'uts03/test/{story_name}.pkl')
dfs = []
for offset in [1, 2, 3, 4]:
    df_offset = df.shift(-offset)
    df_offset.columns = [col + f'_{offset}' for col in df.columns]
    dfs.append(df_offset)
df = pd.concat(dfs, axis=1)  # .dropna()  # would want to dropna here

# load labels
labs = joblib.load(f'labels/test/{story_name}_labels.pkl')

# drop rows with nans
idxs_na = df.isna().sum(axis=1).values > 0
df = df[~idxs_na]
labs = labs[~idxs_na]

print('shapes', df.shape, labs.shape)

shapes (267, 800) (267, 7)


In [3]:
df.head()

,PC0_1,PC1_1,PC2_1,PC3_1,PC4_1,PC5_1,PC6_1,PC7_1,PC8_1,PC9_1,...,PC190_4,PC191_4,PC192_4,PC193_4,PC194_4,PC195_4,PC196_4,PC197_4,PC198_4,PC199_4
off i had been working super,4.420157,-1.524455,1.468934,-0.765363,-0.406178,-0.432300,0.957856,-2.163097,2.114482,1.079989,...,-0.696190,-1.434016,1.072742,-0.515596,0.424500,1.220508,-1.056415,-0.183963,-1.264088,0.105623
hard for a long time,3.817557,-1.606965,0.050622,0.111497,-1.102836,-0.705528,0.780802,-1.671833,1.405054,1.486964,...,-0.738747,-2.651979,-0.564281,-0.483533,0.305728,0.059886,-0.048379,-0.604623,0.192919,-0.272143
i was working on nasa's new horizons,3.351413,-1.143191,-0.858022,-0.184832,-0.846820,-1.270403,0.173373,-1.380383,1.473713,2.004168,...,-0.296728,-2.437131,0.008056,-0.508091,0.628004,1.052285,0.014094,-0.289104,0.099962,-0.313488
mission to pluto and,2.709506,-1.047402,-0.338457,0.106503,-0.631205,-1.043409,0.043512,-1.900928,0.611665,2.571490,...,-0.680306,-0.401569,0.658709,-0.976654,-0.109976,-1.147805,-0.246245,0.253397,0.269854,0.381762
there were always something to do,2.025983,-1.439025,0.155621,-0.476584,0.552912,0.067754,0.906028,-2.012768,0.361617,2.202108,...,-0.317264,-0.937748,-0.428753,-0.941643,0.562629,0.704891,-0.583612,-1.038979,0.000755,-0.571436


In [4]:
labs.head()

,Does the input contain a number?,Is time mentioned in the input?,Does the sentence include dialogue?,Does the input mention or describe high emotional intensity?,Does the sentence mention a specific location?,Is the sentence emotionally positive?,Does the sentence describe a relationship between people?
0,True,True,False,False,True,True,False
1,True,True,False,False,True,True,False
2,True,True,False,False,True,True,False
3,True,True,False,False,True,True,False
4,True,True,False,False,True,True,False
